# Scientific Computing: Diffusion Limited Aggregation and Reaction Diffusion

## Imports

In [1]:
import numpy as np
from IPython.display import HTML

import src.solutions as solutions
import src.visualizations as visualizations

In [2]:
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation


## Diffusion Limited Aggregation

### Parameter Values

In [ ]:
# packed parameters for easy transfer
N = 100
eta = 1
tol = 1e-4
maxiters = 10000
omega = 1.8
grid_indices = np.arange(N*N)
np.random.seed(22)
itertjes = 1500

etas = [0.3, 0.5, 0.7, 1, 1.3]
omegas = np.linspace(1.4, 2, 7)
omegas = np.delete(omegas, -1)
np.round(omegas,1)
print(omegas)

# optimal omegas, determined with experimentation
opp_omm = [1.5, 1.6, 1.6, 1.6, 1.7]
opt_omegas = dict()
for i, e in enumerate(etas):
    opt_omegas[e] = opp_omm[i]

#### Initialization
creating an initial grid by placing object at the bottom of the 2D grid.   
The grid has a source at the top of the grid.   
within SOR, being part of the object is resembled with concentration of 0

In [4]:
object_grid = solutions.place_objects(N)
grid = solutions.initialize_grid(N, object_grid)
iters, grid = solutions.sequential_SOR(grid, tol, maxiters, omega, object_grid)
assert iters < maxiters, f"No convergence for SOR, omega: {omega} "

visualizing diffusion grid along with object (grid is turned 180 degrees)  
object is placed in 0 diffusion zone. 

In [ ]:
# normalized_grid = normalize_concentration(grid)
visualizations.plot_simulation_without_animation(grid, N, object_grid)
solutions.save_grid_to_file(grid)

#### Update initialized grid
run the DLA by solving the Laplace formula of the grid,  
calculating the cell connection probabilities with each cell concentration,  
and by selecting potential new cells joining the object (north, east, south, west) neighbors of the object. 
Update the grid by adding the selected cell to the object and rerunning these steps. 

Visualization of this process is provided here. 

In [6]:
# omega=1.8
# eta = 1
# itertjes = 1500
# iters, grid = solutions.sequential_SOR(grid, tol, maxiters, omega, object_grid)     
# assert iters < maxiters, f"No convergence for SOR, omega: {omega} "
# iter_grid = np.copy(grid)
# object_grid_iter = np.copy(object_grid)
# Sr_pars = (tol, maxiters, omega)
# stencil_iter = solutions.generate_stencil(object_grid_iter)
# seedje = 22
# # print(f"starting animation for DLA, eta: {eta} ")
# ani = animate_1a(iter_grid, stencil_iter, object_grid_iter, grid_indices, eta, seedje, Sr_pars, itertjes)



### Animation of the DLA Process for Different Eta Values

In [ ]:
if omega != 1.7:
    omega = 1.7
    iters, grid = solutions.sequential_SOR(grid, tol, maxiters, omega, object_grid)     
    assert iters < maxiters, f"No convergence for SOR, omega: {omega} "
    
for eta in etas:

    # copy initial grid to save computation time
    iter_grid = np.copy(grid)
    object_grid_iter = np.copy(object_grid)

    # choose precomputed optimal omega for the specific eta
    omega = opt_omegas[eta]

    # pack the parameters for SOR
    Sr_pars = (tol, maxiters, omega)

    # generate the initial neighbors that can be chosen to add to the object
    stencil_iter = solutions.generate_stencil(object_grid_iter)
    seedje = 22
    print(f"starting animation for DLA, eta: {eta}")

    # render visualization (updates are done dynamically)
    ani = visualizations.animate_1a(iter_grid, stencil_iter, object_grid_iter, grid_indices, eta, seedje, Sr_pars)


### Computation of the average number of iterations for a SOR convergence step for Different Omega Values (Run Over 10 Different Grids )

In [ ]:
itertjes = 750
seedje=22

eta_lijstje, grid_lijstje = solutions.optimize_omega_DLA(itertjes, etas, seedje, omegas, grid, object_grid, tol, maxiters, grid_indices)

### Visualization of Average Iterations for Each SOR Convergence Step for Different Omega Values

In [ ]:
best_omegas = visualizations.plot_omega_vs_iterations(eta_lijstje, omegas)
for eetje, ommetje in best_omegas.items():
    print(f"For eta:  {eetje},  the optimal omega is {ommetje} ")

### Visualization of Clusters for Different Eta Values

In [ ]:
visualizations.plot_five_DLA(grid_lijstje, etas)